# Tercer Notebook

El obejtico de este notebook es convertir el subconjunto indexado del dataset COCO (Notebook 01)
al formato requerido por YOLO.

Este notebook:
- Lee los manifests generados (train / val)
- Copia únicamente las imágenes seleccionadas
- Convierte las anotaciones COCO (bbox) a formato YOLO
- Genera la estructura estándar YOLO
- Crea el archivo `data.yaml`


In [1]:
# --- IMPORTS ---
from pathlib import Path
import json
import shutil
from typing import Dict, List, Tuple


In [2]:
# --- RUTAS DEL PROYECTO ---
#Al igual que los notebooks pasados se plantea las carpetas que necesitara para comunicarse entre ellos

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent

COCO_ROOT = Path(r"C:\Users\Johnny\Desktop\IA\data\archive\coco2017")
COCO_TRAIN_IMG_DIR = COCO_ROOT / "train2017"
COCO_VAL_IMG_DIR = COCO_ROOT / "val2017"

ARTIFACTS_DIR = PROJECT_ROOT / "artifacts"
DATASET_INDEX_DIR = ARTIFACTS_DIR / "dataset_index"

PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
YOLO_DATASET_DIR = PROCESSED_DIR / "yolo_dataset"

YOLO_IMAGES_TRAIN = YOLO_DATASET_DIR / "images" / "train"
YOLO_IMAGES_VAL = YOLO_DATASET_DIR / "images" / "val"
YOLO_LABELS_TRAIN = YOLO_DATASET_DIR / "labels" / "train"
YOLO_LABELS_VAL = YOLO_DATASET_DIR / "labels" / "val"

TRAIN_INDEX_PATH = DATASET_INDEX_DIR / "subset_train_coco.json"
VAL_INDEX_PATH = DATASET_INDEX_DIR / "subset_val_coco.json"
META_PATH = DATASET_INDEX_DIR / "subset_meta.json"

print("YOLO_DATASET_DIR:", YOLO_DATASET_DIR)


YOLO_DATASET_DIR: c:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset


In [3]:
# --- VALIDACIONES ---
#validaciones de dtos previos
def assert_exists(p: Path, desc: str) -> None:
    if not p.exists():
        raise FileNotFoundError(f"Falta {desc}: {p}")

assert_exists(TRAIN_INDEX_PATH, "manifest train")
assert_exists(VAL_INDEX_PATH, "manifest val")
assert_exists(META_PATH, "archivo meta")

for d in [
    YOLO_IMAGES_TRAIN, YOLO_IMAGES_VAL,
    YOLO_LABELS_TRAIN, YOLO_LABELS_VAL
]:
    d.mkdir(parents=True, exist_ok=True)

print("Validaciones OK.")


Validaciones OK.


1. Uso carga de data previa en formato COCO  previo a su tranformacion

In [4]:
# --- CARGA DE MANIFESTS ---
#Se necesitan los datos .json creaos en los notebooks previos para el manejo de formato que necesita COCO
with open(TRAIN_INDEX_PATH, "r", encoding="utf-8") as f:
    train_manifest = json.load(f)

with open(VAL_INDEX_PATH, "r", encoding="utf-8") as f:
    val_manifest = json.load(f)

with open(META_PATH, "r", encoding="utf-8") as f:
    meta = json.load(f)

target_classes = meta["target_classes"]
class_to_idx = {name: i for i, name in enumerate(target_classes)}

print("Clases:", target_classes)
#tambien se carga las lables de los objetivos de busqueda de imagenes


Clases: ['car', 'airplane', 'truck']


In [5]:
# --- FUNCIÓN COCO → YOLO (NORMALIZACIÓN DE BBOX) ---
#normalizacion del formato bbox para COCO
def coco_bbox_to_yolo(bbox, img_w, img_h):
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h
    return x_center, y_center, w_norm, h_norm


In [6]:
# --- PROCESAR UN SPLIT (TRAIN / VAL) ---
#se empieza a agenerar el formato que necesita yolo para el preprocesamieinto de las imaganenes
def process_split(manifest: Dict,
                  images_src_dir: Path,
                  images_dst_dir: Path,
                  labels_dst_dir: Path):

    images = {img["id"]: img for img in manifest["images"]}
    anns_by_img = {}

    for ann in manifest["annotations"]:
        anns_by_img.setdefault(ann["image_id"], []).append(ann)

    for img_id, img in images.items():
        img_name = img["file_name"]
        src_img_path = images_src_dir / img_name
        dst_img_path = images_dst_dir / img_name

        if not src_img_path.exists():
            continue

        shutil.copy2(src_img_path, dst_img_path)

        label_path = labels_dst_dir / f"{Path(img_name).stem}.txt"

        with open(label_path, "w", encoding="utf-8") as f:
            for ann in anns_by_img.get(img_id, []):
                cat_id = ann["category_id"]
                cat_name = next(
                    c["name"] for c in manifest["categories"] if c["id"] == cat_id
                )
                cls_idx = class_to_idx[cat_name]

                x, y, w, h = coco_bbox_to_yolo(
                    ann["bbox"], img["width"], img["height"]
                )

                f.write(f"{cls_idx} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")


2. Conversion de la data a formato YOLO

In [7]:
# --- CONVERSIÓN TRAIN ---
#para el entrenmiento se tranforman las imagenes de como un formato json a el formato que YOLo utiliza para el entreno
process_split(
    manifest=train_manifest,
    images_src_dir=COCO_TRAIN_IMG_DIR,
    images_dst_dir=YOLO_IMAGES_TRAIN,
    labels_dst_dir=YOLO_LABELS_TRAIN
)

print("Train convertido.")


Train convertido.


In [8]:
# --- CONVERSIÓN VAL ---
process_split(
    manifest=val_manifest,
    images_src_dir=COCO_VAL_IMG_DIR if COCO_VAL_IMG_DIR.exists() else COCO_TRAIN_IMG_DIR,
    images_dst_dir=YOLO_IMAGES_VAL,
    labels_dst_dir=YOLO_LABELS_VAL
)

print("Val convertido.")


Val convertido.


3. Creacion de archivo yaml, leido por yolo previo a su  entrenamiento, aqui sera el fin del tratamiento de la data previo al entrenamiento 

In [9]:
# --- CREACIÓN DEL data.yaml ---
DATA_YAML_PATH = YOLO_DATASET_DIR / "data.yaml"

data_yaml = f"""
path: {YOLO_DATASET_DIR.as_posix()}
train: images/train
val: images/val

names:
"""

for i, name in enumerate(target_classes):
    data_yaml += f"  {i}: {name}\n"

with open(DATA_YAML_PATH, "w", encoding="utf-8") as f:
    f.write(data_yaml.strip())

print("data.yaml creado en:", DATA_YAML_PATH)


data.yaml creado en: c:\Users\Johnny\Desktop\IA\data\processed\yolo_dataset\data.yaml


In [10]:
# --- RESUMEN FINAL ---
#numero de imagenes y labels que seran usados para el train y nueva data
def count_files(p: Path):
    return len(list(p.glob("*")))

print("Imágenes train:", count_files(YOLO_IMAGES_TRAIN))
print("Labels train:", count_files(YOLO_LABELS_TRAIN))
print("Imágenes val:", count_files(YOLO_IMAGES_VAL))
print("Labels val:", count_files(YOLO_LABELS_VAL))


Imágenes train: 2341
Labels train: 2341
Imágenes val: 550
Labels val: 550
